In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'01.16.2022'

In [4]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FanDuel_Data/PlayerPool'

shutil.move(downloads + "/"+ "FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv",
           contest_data_loc + "/"+ "FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
fanduel_file = shutil.copy("FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv", "FanDuelSalaries_"+ contest + str(today)+".csv")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/FanDuelSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

FileNotFoundError: [Errno 2] No such file or directory: 'FanDuel-NBA-2022 ET-01 ET-16 ET-70321-players-list.csv'

In [53]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [54]:
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FanDuel_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,70321-55062,C,Nikola,Nikola Jokic,Jokic,55.477776,36,10900,UTA@DEN,DEN,UTA,NaN,NaN,NaN,NaN,NaN,C
1,70321-9524,PG,Stephen,Stephen Curry,Curry,44.125642,39,10600,GS@MIN,GS,MIN,O,Hand,NaN,NaN,NaN,PG
2,70321-58462,C,Karl-Anthony,Karl-Anthony Towns,Towns,44.594287,35,9800,GS@MIN,MIN,GS,NaN,NaN,NaN,NaN,NaN,C
3,70321-40203,C,Rudy,Rudy Gobert,Gobert,42.510811,37,8600,UTA@DEN,UTA,DEN,NaN,NaN,NaN,NaN,NaN,C
4,70321-66113,SG/PG,Donovan,Donovan Mitchell,Mitchell,40.460526,38,8400,UTA@DEN,UTA,DEN,NaN,NaN,NaN,NaN,NaN,PG/SG


In [60]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FanDuel_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Roster Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [61]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Roster Position,Id
0,Jordan Poole,GSW,MIN,33.0,PG/SG,"4,900",34.4,9.8,43.4,67.2,MAIN,Jordan Poole,PG/SG,70321-110322
1,Nikola Jokic,DEN,UTA,34.0,C,"10,900",58.0,9.4,66.9,58.4,MAIN,Nikola Jokic,C,70321-55062
2,Terence Davis,SAC,HOU,25.0,SG/SF,"3,500",25.6,6.6,34.9,53.4,MAIN,Terence Davis,SG/SF,70321-116340
3,De'Aaron Fox,SAC,HOU,35.5,PG,"8,000",43.6,6.6,53.8,58.6,MAIN,De'Aaron Fox,PG,70321-80810
4,Christian Wood,HOU,SAC,31.0,PF/C,"7,300",39.5,5.3,47.9,64.4,MAIN,Christian Wood,PF/C,70321-42202


In [62]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Roster Position,Id
0,Jordan Poole,GSW,MIN,33.0,PG/SG,4900.0,34.4,9.8,43.4,67.2,MAIN,Jordan Poole,"[PG, SG]",70321-110322
1,Nikola Jokic,DEN,UTA,34.0,C,10900.0,58.0,9.4,66.9,58.4,MAIN,Nikola Jokic,[C],70321-55062
2,Terence Davis,SAC,HOU,25.0,SG/SF,3500.0,25.6,6.6,34.9,53.4,MAIN,Terence Davis,"[SG, SF]",70321-116340
3,De'Aaron Fox,SAC,HOU,35.5,PG,8000.0,43.6,6.6,53.8,58.6,MAIN,De'Aaron Fox,[PG],70321-80810
4,Christian Wood,HOU,SAC,31.0,PF/C,7300.0,39.5,5.3,47.9,64.4,MAIN,Christian Wood,"[PF, C]",70321-42202


In [63]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
os.system("FD_final_PlayerPool_ETR_"+str(today)+".xlsx")

0

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [64]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [65]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [66]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [67]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['FD Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'PG') <= 2
prob += get_position_sum(player_vars, df, 'PG') >= 2
prob += get_position_sum(player_vars, df, 'SG') <= 2
prob += get_position_sum(player_vars, df, 'SG') >= 2
# prob += get_position_sum(player_vars, df, 'G') <= 2
# prob += get_position_sum(player_vars, df, 'G') >= 2
prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
# prob += get_position_sum(player_vars, df, 'F') <= 2
# prob += get_position_sum(player_vars, df, 'F') >= 2
prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 2
# prob += get_position_sum(player_vars, df, 'UTIL') <= 2

#prob += get_position_sum(player_vars, df, 'PG') <= 3
#prob += get_position_sum(player_vars, df, 'PG') >= 1
#prob += get_position_sum(player_vars, df, 'SG') <= 3
#prob += get_position_sum(player_vars, df, 'SG') >= 1
#prob += get_position_sum(player_vars, df, 'G') == 1
#prob += get_position_sum(player_vars, df, 'SF') <= 3
#prob += get_position_sum(player_vars, df, 'SF') >= 1
#prob += get_position_sum(player_vars, df, 'PF') <= 3
#prob += get_position_sum(player_vars, df, 'PF') >= 1
#prob += get_position_sum(player_vars, df, 'F') == 1
#prob += get_position_sum(player_vars, df, 'C') <= 2
#prob += get_position_sum(player_vars, df, 'C') >= 1
#prob += get_position_sum(player_vars, df, 'UTIL') == 1

In [68]:
# total salary constraint

prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

In [69]:
# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [70]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [71]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['FD Position'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

C Nikola Jokic DEN 10900.0 58.0
PG De'Aaron Fox SAC 8000.0 43.6
SF Andrew Wiggins GSW 5800.0 31.6
PF Aaron Gordon DEN 5200.0 24.9
PF Jarred Vanderbilt MIN 6300.0 27.5
SG Klay Thompson GSW 5800.0 25.4
PG Mike Conley UTA 6500.0 27.0
SG Josh Christopher HOU 3700.0 15.3
SF Jonathan Kuminga GSW 4400.0 14.6


(56600.0, 267.90000000000003)

In [52]:
df.dtypes

Player              object
Team                object
Opp                 object
Minutes            float64
FD Position         object
FD Salary          float64
FD Points          float64
FD Value           float64
FDCeiling          float64
FD Own             float64
FD Slate            object
Nickname            object
Roster Position     object
Id                  object
dtype: object